In [28]:
import torch
import torch.nn.functional as F
import matplotlib.pyplot as plt # for making figures

In [29]:
words = open("data.txt", 'r').read().splitlines()

In [30]:
words[:8]
len(words)

370104

In [31]:
# build the vocabulary of characters and mappings to/from integers
chars = sorted(list(set(''.join(words))))
stoi = {s:i+1 for i,s in enumerate(chars)}
stoi['.'] = 0
itos = {i:s for s,i in stoi.items()}
print(itos)

{1: 'a', 2: 'b', 3: 'c', 4: 'd', 5: 'e', 6: 'f', 7: 'g', 8: 'h', 9: 'i', 10: 'j', 11: 'k', 12: 'l', 13: 'm', 14: 'n', 15: 'o', 16: 'p', 17: 'q', 18: 'r', 19: 's', 20: 't', 21: 'u', 22: 'v', 23: 'w', 24: 'x', 25: 'y', 26: 'z', 0: '.'}


In [35]:
# build the dataset

block_size = 3 # context length: how many characters do we take to predict the next one?
X, Y = [], []
for w in words:
  #print(w)
  context = [0] * block_size
  for ch in w + '.':
    ix = stoi[ch]
    X.append(context)
    Y.append(ix)
    #print(''.join(itos[i] for i in context), '--->', itos[ix])
    context = context[1:] + [ix] # crop and append
  
X = torch.tensor(X)
Y = torch.tensor(Y)

In [33]:
X.shape, X.dtype, Y.shape, Y.dtype

(torch.Size([3864799, 3]), torch.int64, torch.Size([3864799]), torch.int64)

In [36]:
# build the dataset
block_size = 3 # context length: how many characters do we take to predict the next one?

def build_dataset(words):  
  X, Y = [], []
  for w in words:

    #print(w)
    context = [0] * block_size
    for ch in w + '.':
      ix = stoi[ch]
      X.append(context)
      Y.append(ix)
      #print(''.join(itos[i] for i in context), '--->', itos[ix])
      context = context[1:] + [ix] # crop and append

  X = torch.tensor(X)
  Y = torch.tensor(Y)
  print(X.shape, Y.shape)
  return X, Y

import random
random.seed(42)
random.shuffle(words)
n1 = int(0.8*len(words))
n2 = int(0.9*len(words))

Xtr, Ytr = build_dataset(words[:n1])
Xdev, Ydev = build_dataset(words[n1:n2])
Xte, Yte = build_dataset(words[n2:])

torch.Size([3092569, 3]) torch.Size([3092569])
torch.Size([385429, 3]) torch.Size([385429])
torch.Size([386801, 3]) torch.Size([386801])


In [91]:
C = torch.arange((27, 2))

test_X = torch.arange(0,9).view(3,3)

In [92]:
emb = C[test_X]
emb

tensor([[[ 1.6732, -0.6163],
         [-1.0377,  0.9136],
         [-0.6292,  0.7676]],

        [[ 0.2970, -0.5530],
         [-0.8934, -1.5825],
         [-0.8287,  0.1234]],

        [[-0.1494, -0.6206],
         [ 1.1535, -1.3162],
         [ 1.4858, -0.2310]]])

In [89]:
emb = C[test_X].view(-1,6)
emb

tensor([[ 0.4460, -0.7141, -1.6012,  0.2673, -0.8208, -0.6610],
        [-1.6839,  0.6240, -1.4781,  0.7850,  0.2678, -0.0235],
        [-0.2124, -3.2374, -0.6560, -0.7430,  0.1680, -0.6417]])

In [40]:
W1 = torch.randn((6, 100))
b1 = torch.randn(100)

In [45]:
emb.view(-1,6).shape

torch.Size([3864799, 6])

In [44]:
h = torch.tanh(emb.view(-1, 6) @ W1 + b1)
h

tensor([[-0.9989, -0.6096,  0.9989,  ..., -0.9930,  0.4671,  0.8433],
        [-0.9924, -0.5334,  0.9993,  ..., -0.9950,  0.0756,  0.9514],
        [-0.9989, -0.6096,  0.9989,  ..., -0.9930,  0.4671,  0.8433],
        ...,
        [-0.9998,  0.9281,  0.9986,  ..., -0.9774,  0.8828,  0.3890],
        [-0.9997,  0.9892,  0.9337,  ..., -0.9754,  0.9717, -0.4691],
        [-0.9948,  0.7360,  1.0000,  ..., -0.9997, -0.1761,  0.9997]])

In [ ]:
W2 = torch.randn((100, 27))
b2 = torch.randn(27)